In [15]:
import os
from ase.build import diamond100
from ase.io import write
from ase.build import bulk
import numpy as np

## Silicon (100) Thin Film

In [16]:
# Layer numbers (thickness) for Si(100) slab
Nz_list = [8, 12, 16, 24, 40, 80]

# Create output directory
os.makedirs('Structure', exist_ok=True)

In [17]:
for Nz in Nz_list:
    # 1) Build a Si(100) slab with Nz layers and 10 Å vacuum in z-direction
    slab = diamond100('Si', size=(2, 2, Nz), vacuum=10.0)

    # 2) Get atomic positions and extract z-coordinates
    positions = slab.get_positions()
    z_coords = positions[:, 2]

    # 3) Determine top and bottom layer atoms based on z-coordinates
    zmax = z_coords.max()/10
    zmin = z_coords.min()/10
    top_indices = [4*Nz-4, 4*Nz-3, 4*Nz-2, 4*Nz-1]
    bottom_indices = [0, 1, 2, 3]

    # 4) Apply surface reconstruction by displacing top atoms
    for i, idx in enumerate(top_indices):
        y_offsets = {0: 0.5, 1: 0.9, 2: -0.9, 3: -0.5}
        positions[idx, 1] += y_offsets[i % 4]
        positions[idx, 2] -= 0.3

    # 5) Apply reconstruction to bottom atoms
    for i, idx in enumerate(bottom_indices):
        y_offsets = {0: 0.5, 1: 0.9, 2: -0.9, 3: -0.5}
        positions[idx, 1] += y_offsets[i % 4]
        positions[idx, 2] -= 0.3

    # 6) Set updated atomic positions
    slab.set_positions(positions)

    # 7) Save the structure to .xyz and .cif
    write(f'Structure/Si_film_{Nz/4*0.543:.2f}nm.xyz', slab)
    write(f'Structure/Si_film_{Nz/4*0.543:.2f}nm.cif', slab)

    print(f"Generated Si_film with {len(slab)} atoms, thickness ~ {Nz/4*0.543:.2f} nm")


Generated Si_film with 32 atoms, thickness ~ 1.09 nm
Generated Si_film with 48 atoms, thickness ~ 1.63 nm
Generated Si_film with 64 atoms, thickness ~ 2.17 nm
Generated Si_film with 96 atoms, thickness ~ 3.26 nm
Generated Si_film with 160 atoms, thickness ~ 5.43 nm
Generated Si_film with 320 atoms, thickness ~ 10.86 nm


## Silicon Nanowires

In [18]:
# Adjustable parameters
NxNy_list        = [2, 3, 4, 6, 10, 20]   # Cross-sectional cell count (Nx = Ny)
vacuum_thickness = 10.0                  # Vacuum in x and y directions (Å)
a0               = 5.431                 # Silicon lattice constant (Å)

# Create base unit cell
ucell = bulk('Si', 'diamond', a=a0, cubic=True)

In [19]:
for N in NxNy_list:
    # 1) Replicate unit cell in x and y to form Nx x Nx x 1 nanowire
    wire = ucell.repeat((N, N, 1))

    # 2) Add vacuum in x and y directions, keep periodic in z
    wire.center(vacuum=vacuum_thickness, axis=(0, 1))
    wire.set_pbc([False, False, True])

    # 3) Delete first and last atoms
    del wire[[-1, 0]]

    # 4) Estimate cross-section in nm and save structure
    width_nm = N * a0 / 10.0  # convert Å to nm
    fname = f"Si_wire_{width_nm:.2f}nm"
    write(f"Structure/{fname}.xyz", wire)
    write(f"Structure/{fname}.cif", wire)

    print(f"Generated {fname}: {len(wire)} atoms, cross-section ≈ {width_nm:.2f} × {width_nm:.2f} nm, periodic along z")


Generated Si_wire_1.09nm: 30 atoms, cross-section ≈ 1.09 × 1.09 nm, periodic along z
Generated Si_wire_1.63nm: 70 atoms, cross-section ≈ 1.63 × 1.63 nm, periodic along z
Generated Si_wire_2.17nm: 126 atoms, cross-section ≈ 2.17 × 2.17 nm, periodic along z
Generated Si_wire_3.26nm: 286 atoms, cross-section ≈ 3.26 × 3.26 nm, periodic along z
Generated Si_wire_5.43nm: 798 atoms, cross-section ≈ 5.43 × 5.43 nm, periodic along z
Generated Si_wire_10.86nm: 3198 atoms, cross-section ≈ 10.86 × 10.86 nm, periodic along z


## Silicon Quantum Dots

In [20]:
# Adjustable parameters
NxNy_list        = [2, 3, 4, 6, 10, 20]   # Number of unit cells in x, y, z directions
vacuum_thickness = 10.0                  # Vacuum in all directions (Å)
a0               = 5.431                 # Silicon lattice constant (Å)

# Create base unit cell again
ucell = bulk('Si', 'diamond', a=a0, cubic=True)

In [21]:

for N in NxNy_list:
    # 1) Create a cube of size N×N×N
    dot = ucell.repeat((N, N, N))

    # 2) Add vacuum in all directions, making it non-periodic
    dot.center(vacuum=vacuum_thickness, axis=(0, 1, 2))
    dot.set_pbc([False, False, False])

    # 3) Estimate dot dimensions and save
    width_nm = N * a0 / 10.0  # convert Å to nm
    fname = f"Si_dot_{width_nm:.2f}nm"
    write(f"Structure/{fname}.xyz", dot)
    write(f"Structure/{fname}.cif", dot)

    print(f"Generated {fname}: {len(dot)} atoms, size ≈ {width_nm:.2f} × {width_nm:.2f} × {width_nm:.2f} nm")

Generated Si_dot_1.09nm: 64 atoms, size ≈ 1.09 × 1.09 × 1.09 nm
Generated Si_dot_1.63nm: 216 atoms, size ≈ 1.63 × 1.63 × 1.63 nm
Generated Si_dot_2.17nm: 512 atoms, size ≈ 2.17 × 2.17 × 2.17 nm
Generated Si_dot_3.26nm: 1728 atoms, size ≈ 3.26 × 3.26 × 3.26 nm
Generated Si_dot_5.43nm: 8000 atoms, size ≈ 5.43 × 5.43 × 5.43 nm
Generated Si_dot_10.86nm: 64000 atoms, size ≈ 10.86 × 10.86 × 10.86 nm
